# Processing the given time series data to create and store the training, validation and test datasets for supervised model building

This notebook is created as part of the coursera course "How to win data science competitions" Final project.

References:
* https://towardsdatascience.com/using-gradient-boosting-for-time-series-prediction-tasks-600fac66a5fc

## Lets look at the test dataset on which we have to make the prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/CourseraProject/data/test.csv')
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


## Q1) What is the shape of the test dataset

In [ ]:
df_test.shape

(214200, 3)

So there are around 2 Lakh+ entries in the test dataset

## Q2) What are the names of the columns?

In [ ]:
df_test.columns

Index(['ID', 'shop_id', 'item_id'], dtype='object')

A ID indicates a particular shop_id and item_id pair combination.

### Q3) How many unique shop ids are there?

In [ ]:
df_test['shop_id'].value_counts().count()

42

There are 42 unique shop ids in the test dataset

### Q4) How many unique item ids are there?

In [ ]:
df_test['item_id'].value_counts().count()

5100

There are 5100 unique item ids in the test dataset

In [ ]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [ ]:
# How the test set was created
from itertools import product
test_comb = list(product(*[df_test.shop_id.unique(),df_test.item_id.unique()]))
len(test_comb)

214200

The test set was generated by cartesian product of unique values from shop_id and item_id columns of the test dataset

In [ ]:
#Lets read the main training dataset
df_train = pd.read_csv('/content/drive/My Drive/CourseraProject/data/sales_train.csv')
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [ ]:
df_train.shape

(2935849, 6)

In [ ]:
df_train.date_block_num.nunique()

34

In [ ]:
# How many unique elements in shop_id and item_id of sales train?
print(df_train.shop_id.nunique())
print(df_train.item_id.nunique())

60
21807


Since the test data is generated with combination of shops and items, we have to restructure train data to match the test data generation. 

In [ ]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = []
for block_num in df_train['date_block_num'].unique():
    cur_shops = df_train.loc[df_train['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = df_train.loc[df_train['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
print(grid.shape)
grid.head()

(10913850, 3)


,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [ ]:
print(grid.shape[0]/df_train.shape[0])

3.717442552392851


The training data is smaller than all possible combinations calculated using shop_id, item_id columns of the training data

In [ ]:
df_test.merge(df_train,how='left',on=['shop_id','item_id']).isnull().sum()

ID                     0
shop_id                0
item_id                0
date              102796
date_block_num    102796
item_price        102796
item_cnt_day      102796
dtype: int64

In [ ]:
df_test.merge(df_train,how='left',on=['shop_id','item_id']).head()

,ID,shop_id,item_id,date,date_block_num,item_price,item_cnt_day
0,0,5,5037,21.09.2014,20.0,2599.0,1.0
1,0,5,5037,29.11.2014,22.0,2599.0,1.0
2,0,5,5037,28.12.2014,23.0,1999.0,1.0
3,0,5,5037,20.12.2014,23.0,1999.0,1.0
4,0,5,5037,02.01.2015,24.0,1999.0,1.0


In [ ]:
# Removing the outliers from the item_price and item_cnt_day
df_train = df_train[df_train.item_price<100000]
df_train = df_train[df_train.item_cnt_day<=900]

In [ ]:
df_train.shape

(2935846, 6)

3 rows are dropped from the df_train

# Let us create the final train dataset.

In [ ]:
# Test predictions require item_cnt prediction for a month so in training it should also be in monthly format
df_train_m = df_train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
# To mimic the test data generation process..
df_train_m = pd.merge(grid,df_train_m,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)

In [ ]:
df_train_m.head()

,shop_id,item_id,date_block_num,item_cnt_day,item_price
0,59,22154,0,1.0,999.0
1,59,2552,0,0.0,0.0
2,59,2554,0,0.0,0.0
3,59,2555,0,0.0,0.0
4,59,2564,0,0.0,0.0


In [ ]:
# Lets read remaining files
df_item_cat = pd.read_csv('/content/drive/My Drive/CourseraProject/data/item_categories.csv')
df_item = pd.read_csv('/content/drive/My Drive/CourseraProject/data/items.csv')
df_shops = pd.read_csv('/content/drive/My Drive/CourseraProject/data/shops.csv')

In [ ]:
df_item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
df_item.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [ ]:
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [ ]:
# df_shop is not necessary to be merged with training dataframe as its just shop name and respective ids
# lets merge df_item and df_item_cat with df_train_m
df_train_m = pd.merge(df_train_m,df_item,on=['item_id'],how='left')
df_train_m = pd.merge(df_train_m,df_item_cat,on=['item_category_id'], how='left')
df_train_m.head()

,shop_id,item_id,date_block_num,item_cnt_day,item_price,item_name,item_category_id,item_category_name
0,59,22154,0,1.0,999.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,59,2552,0,0.0,0.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,59,2554,0,0.0,0.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,59,2555,0,0.0,0.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,59,2564,0,0.0,0.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


In [ ]:
for type_id in ['item_id','shop_id','item_category_id']:
    for column_id, agg_func, func_name in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:

        mean_df = df_train_m.groupby([type_id,'date_block_num']).agg(agg_func).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+func_name+'_'+column_id, type_id,'date_block_num']
        
        df_train_m = pd.merge(df_train_m,mean_df,on=['date_block_num', type_id],how='left')

We append test data into train data so we can create lag features on them.

In [ ]:
temp_test = df_test.copy()
temp_test['date_block_num'] = 34
temp_test.drop('ID', axis=1, inplace=True)

temp_test = temp_test.merge(df_item,on=['item_id'],how='left')
temp_test = temp_test.merge(df_item_cat,on=['item_category_id'], how='left')
temp_test.drop('item_name', axis=1, inplace=True)
temp_test.head()

,shop_id,item_id,date_block_num,item_category_id,item_category_name
0,5,5037,34,19,Игры - PS3
1,5,5320,34,55,Музыка - CD локального производства
2,5,5233,34,19,Игры - PS3
3,5,5232,34,23,Игры - XBOX 360
4,5,5268,34,20,Игры - PS4


In [ ]:
df_train_m.drop('item_name', axis=1, inplace=True)
df_train_m = pd.concat([df_train_m,temp_test], axis=0, ignore_index=True)


In [ ]:
df_train_m.head()

,shop_id,item_id,date_block_num,item_cnt_day,item_price,item_category_id,item_category_name,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price,shop_id_sum_item_cnt_day,shop_id_avg_item_cnt_day,item_category_id_avg_item_price,item_category_id_sum_item_cnt_day,item_category_id_avg_item_cnt_day
0,59,22154,0,1.0,999.0,37,Кино - Blu-Ray,310.800000,18.0,0.400000,97.974472,2017.0,0.248552,63.014719,6094.0,0.199738
1,59,2552,0,0.0,0.0,58,Музыка - Винил,19.977778,0.0,0.000000,97.974472,2017.0,0.248552,54.898665,287.0,0.043386
2,59,2554,0,0.0,0.0,58,Музыка - Винил,37.978889,1.0,0.022222,97.974472,2017.0,0.248552,54.898665,287.0,0.043386
3,59,2555,0,0.0,0.0,56,Музыка - CD фирменного производства,48.837778,2.0,0.044444,97.974472,2017.0,0.248552,38.267019,268.0,0.049630
4,59,2564,0,0.0,0.0,59,Музыка - Музыкальное видео,36.400000,5.0,0.111111,97.974472,2017.0,0.248552,35.593867,701.0,0.093280


In [ ]:
df_train_m = df_train_m.drop('item_category_name',axis=1)

We create lags on 10 features. 9 features are from mean encoding and 1 feature is the item_cnt_day.

After several tries, we found that month lag intervals of 1, 2, 3, 4, 5, 6, 9 and 12 give best score in the leaderboard. Take note that 16GB of ram is unable to compute such large number of interval. We need at least 64GB of ram. 

In [ ]:
lag_variables  = ['item_id_avg_item_price',
'item_id_sum_item_cnt_day',
'item_id_avg_item_cnt_day',
'shop_id_avg_item_price',
'shop_id_sum_item_cnt_day',
'shop_id_avg_item_cnt_day',
'item_category_id_avg_item_price',
'item_category_id_sum_item_cnt_day',
'item_category_id_avg_item_cnt_day',
'item_cnt_day']

#Limited by computation resource
lags = [1,2]
#lags = [1, 2, 3, 4, 5, 6, 12]

for lag in lags:
    df_train_m_new = df_train_m.copy()
    df_train_m_new.date_block_num+=lag
    df_train_m_new = df_train_m_new[['date_block_num','shop_id','item_id']+lag_variables]
    df_train_m_new.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    df_train_m = df_train_m.merge(df_train_m_new,on=['date_block_num','shop_id','item_id'] ,how='left')

In [ ]:
df_train_m.describe()

,shop_id,item_id,date_block_num,item_cnt_day,item_price,item_category_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price,shop_id_sum_item_cnt_day,shop_id_avg_item_cnt_day,item_category_id_avg_item_price,item_category_id_sum_item_cnt_day,item_category_id_avg_item_cnt_day,item_id_avg_item_price_lag_1,item_id_sum_item_cnt_day_lag_1,item_id_avg_item_cnt_day_lag_1,shop_id_avg_item_price_lag_1,shop_id_sum_item_cnt_day_lag_1,shop_id_avg_item_cnt_day_lag_1,item_category_id_avg_item_price_lag_1,item_category_id_sum_item_cnt_day_lag_1,item_category_id_avg_item_cnt_day_lag_1,item_cnt_day_lag_1,item_id_avg_item_price_lag_2,item_id_sum_item_cnt_day_lag_2,item_id_avg_item_cnt_day_lag_2,shop_id_avg_item_price_lag_2,shop_id_sum_item_cnt_day_lag_2,shop_id_avg_item_cnt_day_lag_2,item_category_id_avg_item_price_lag_2,item_category_id_sum_item_cnt_day_lag_2,item_category_id_avg_item_cnt_day_lag_2,item_cnt_day_lag_2
count,1.112805e+07,1.112805e+07,1.112805e+07,1.091385e+07,1.091385e+07,1.112805e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,8.605461e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06,7.953134e+06
mean,3.119598e+01,1.130368e+04,1.533958e+01,3.339826e-01,1.165788e+02,4.494411e+01,1.165788e+02,1.563809e+01,3.339826e-01,1.165788e+02,2.375230e+03,3.339826e-01,1.165788e+02,8.501826e+03,3.339826e-01,1.397048e+02,1.904271e+01,4.066517e-01,1.167099e+02,2.390998e+03,3.348756e-01,1.217431e+02,8.750132e+03,3.512910e-01,4.075280e-01,1.426280e+02,1.973502e+01,4.210480e-01,1.157925e+02,2.410998e+03,3.361374e-01,1.209647e+02,8.838638e+03,3.525775e-01,4.229887e-01
std,1.735381e+01,6.210930e+03,9.760396e+00,3.334916e+00,6.518409e+02,1.514009e+01,4.436272e+02,9.621937e+01,2.041777e+00,6.043432e+01,1.952808e+03,2.644711e-01,3.204518e+02,8.973466e+03,1.638926e+00,4.914868e+02,1.071965e+02,2.273987e+00,6.006481e+01,1.952238e+03,2.637254e-01,3.378542e+02,9.017931e+03,1.812894e+00,3.578130e+00,4.940315e+02,1.097803e+02,2.328683e+00,5.916657e+01,1.962018e+03,2.644368e-01,3.346894e+02,9.052480e+03,1.851287e+00,3.619428e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01,-4.888889e-01,2.329451e-02,-1.000000e+00,-1.966568e-04,2.600000e-01,-1.000000e+00,-2.272727e-02,4.444444e-03,-2.200000e+01,-4.888889e-01,2.329451e-02,2.000000e+00,3.025261e-04,4.130435e-01,1.000000e+00,7.751938e-03,-2.200000e+01,4.444444e-03,-2.200000e+01,-4.888889e-01,2.329451e-02,2.000000e+00,3.025261e-04,4.130435e-01,1.000000e+00,1.785714e-02,-2.200000e+01
25%,1.600000e+01,5.947000e+03,7.000000e+00,0.000000e+00,0.000000e+00,3.700000e+01,9.652174e+00,2.000000e+00,3.921569e-02,7.951914e+01,1.228000e+03,1.837063e-01,2.973899e+01,1.077000e+03,1.578334e-01,1.328889e+01,2.000000e+00,4.651163e-02,7.945678e+01,1.243000e+03,1.853114e-01,2.993934e+01,1.162000e+03,1.714948e-01,0.000000e+00,1.436957e+01,3.000000e+00,6.000000e-02,7.920055e+01,1.251000e+03,1.863496e-01,2.993934e+01,1.169000e+03,1.714948e-01,0.000000e+00
50%,3.000000e+01,1.138800e+04,1.500000e+01,0.000000e+00,0.000000e+00,4.000000e+01,2.706250e+01,4.000000e+00,8.695652e-02,1.026577e+02,1.807000e+03,2.560714e-01,4.557390e+01,5.959000e+03,2.254803e-01,3.481395e+01,6.000000e+00,1.276596e-01,1.026263e+02,1.816000e+03,2.566306e-01,4.569278e+01,6.088000e+03,2.285004e-01,0.000000e+00,3.588000e+01,6.000000e+00,1.304348e-01,1.020176e+02,1.833000e+03,2.573808e-01,4.569278e+01,6.181000e+03,2.285004e-01,0.000000e+00
75%,4.700000e+01,1.659200e+04,2.300000e+01,0.000000e+00,0.000000e+00,5.500000e+01,7.740851e+01,1.100000e+01,2.391304e-01,1.402056e+02,2.683000e+03,3.571321e-01,1.110535e+02,1.283400e+04,2.876345e-01,9.862222e+01,1.400000e+01,3.043478e-01,1.399021e+02,2.700000e+03,3.564936e

In [ ]:
df_train_m.shape

(11128050, 35)

In [ ]:
for feat in df_train_m.columns:
    if 'item_cnt' in feat:
        df_train_m[feat]=df_train_m[feat].fillna(0)
    elif 'item_price' in feat:
        df_train_m[feat]=df_train_m[feat].fillna(df_train_m[feat].median())

In [ ]:
cols_to_drop = lag_variables[:-1] + ['item_price']
df_train_m = df_train_m[df_train_m['date_block_num']>12]

# Validation

We use holdout scheme for cross validation. We use sales month from 12 to 32 for training, month 33 for validation and month 34 for testing. 

In [ ]:
X_train = df_train_m[df_train_m['date_block_num']<33].drop(cols_to_drop, axis=1)
X_cv =  df_train_m[df_train_m['date_block_num']==33].drop(cols_to_drop, axis=1)
X_test = df_train_m[df_train_m['date_block_num']==34].drop(cols_to_drop, axis=1)

# Reference from discussion

From discussion in Kaggle forum, we applied this useful trick, which clip the item_cnt_day within 40 instead of 20. In this way, there are more predictions on item_cnt_day = 20.

In [ ]:
X_train['item_cnt_day'].clip(0,40, inplace=True)

X_cv['item_cnt_day'].clip(0,40, inplace=True)

X_test['item_cnt_day'].clip(0,40, inplace=True)

In [ ]:
X_train.to_csv('drive/My Drive/CourseraProject/data/X_train.csv', index=False)
X_cv.to_csv('drive/My Drive/CourseraProject/data/X_cv.csv', index=False)
X_test.to_csv('drive/My Drive/CourseraProject/data/X_test.csv', index=False)